In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
import itertools as it
import scipy.linalg
import time
import copy as cp

# Theory parameters

### Hamiltonian of the Schwinger model:
$$H = \frac{i}{2a}\sum_x{\psi^\dagger(x)U(x)\psi(x+1)+{\rm h.c.}} + m\sum_x{(-1)^x\psi^\dagger(x)\psi(x)} + \frac{ag^2}{2}\sum_x{E(x)^2}$$
These are the hopping term, the mass term and the electric term in the electric basis. $\psi$, $\psi^\dagger$ are fermionic operators and $U$, $E$ are bosonic.

In [2]:
N = 4          # number of sites
a = 1          # lattice spacing

Nphys = N//2   # number of physical sites
mf = 0.1       # fermion mass
g2 = 1.0       # electric coupling

In [3]:
Lamb = 1       # truncation on bosonic dof
Hilbert_full = (2*(2*Lamb+1))**N
print("Size of full Hilbert space =", Hilbert_full)

Size of full Hilbert space = 1296


# Physical Hilbert space

The shape of the configurations = (2, N). The first(second) row represents fermionic(bosonic) degrees of freedom. The fermionic dofs can be 0 or 1, while the bosonic dofs can take integer values between $-\Lambda$ and $\Lambda$. The first task is to find out which configurations are physical (obey Gauss' law). The Gauss' law in Schwinger model is:

$$ G(x) = E(x) - E(x-1) + \psi^\dagger(x)\psi(x) - \frac{1-(-1)^x}{2} $$
$$ G(x)|\psi_{phys}>=0 $$

Note: in 1 spacial dimension the bosonic dofs can be fixed by the fermionic ones, so instead of running through all possible configurations one can have more efficient algorithm. Here we keep it general.

In [4]:
def Gauss(config): # Gauss' law
    L = len(config[0])
    out = True
    for x in range(L): # Here we use PBC
        # implement the Gauss' law
        
        
        
    return out

In [5]:
config_all = [] # configurations in full Hilbert space
config_phys = [] # configurations in physical Hilbert space
for fer in it.product((0,1), repeat=N): # fermionic dof = 0 or 1
    for bos in it.product(range(-Lamb, Lamb + 1), repeat=N): # bosonic dof runs from -\Lambda to Lambda
        config = [list(fer), list(bos)]
        config_all.append(config)
        if Gauss(config):
            config_phys.append(config) # collect the physical ones

In [6]:
Hilbert_phys = len(config_phys)
print("Size of physical Hilbert space =", Hilbert_phys)

Size of physical Hilbert space = 13


# Building Hamiltonian

After the physical configurations are identified, one can construct the Hamiltoninan in the physical Hilbert space. In electric basis, the mass term and the electric term are diagonal and are easy to implement. Here you need to complete the construction of the hopping term.

### Operations on the configurations

In [7]:
def fer_sign(config, x): # general fermionic signs, may not be needed here
    return((-1)**np.sum(config[:x]))

def mass_term(config): 
    out = 0
    for i in range(len(config[0])):
        out += config[0][i]*(-1)**(i) # staggered fermion's excitation
    return out

def electric_term(config): 
    out = 0
    for i in range(len(config[1])):
        out += (config[1][i])**2 # quadratic electric term
    return out

def hopping_term(config): 
    out = []
    for i in range(lwn(config[1])):
        # implement the hopping term
        
        
        
        
        
    return out

In [8]:
HI = np.zeros([Hilbert_phys, Hilbert_phys], dtype = complex)
for i in range(Hilbert_phys):
    out = hopping_term(config_phys[i])
    if out != []:
        for res in out:
            idx = config_phys.index(res[1])
            HI[idx,i] += 0.5j/a*res[0]

HM = np.zeros([Hilbert_phys, Hilbert_phys], dtype = complex)
for i in range(Hilbert_phys):
    HM[i,i] += mass_term(config_phys[i])

HE = np.zeros([Hilbert_phys, Hilbert_phys], dtype = complex)
for i in range(Hilbert_phys):
    HE[i,i] += electric_term(config_phys[i])

H = HI + HI.conj().T + mf*HM + 0.5*g2/a*HE

As a consistency check with 1803.03326, set $N=4$ with the parameters below and look for the ground state $E_0 = −1.011810$. There may be a small error due to the choice of $\Lambda$

In [9]:
'''
mf = 0.1
x = 0.6
g2 = 1/x
mff = mf/(x*2) # These are the 
fac = 2/g2
H = (HI + HI.conj().T + mff*HM + 0.5*g2*HE)*fac 

'''

'\nmf = 0.1\nx = 0.6\ng2 = 1/x\nmff = mf/(x*2) # These are the \nfac = 2/g2\nH = (HI + HI.conj().T + mff*HM + 0.5*g2*HE)*fac \n\n'

# Spectrum

In [10]:
def is_hermitian(H):
    HC = H.conj().T
    return(np.all(np.isclose(HC, H)))

def eigen(H):
    Hw, Hv =  scipy.linalg.eigh(H)
    idx = Hw.argsort() #small to large
    Hw = Hw[idx].real
    Hv = Hv.T
    Hv = Hv[idx]
    return Hw, Hv

In [11]:
print(H.shape)
print("Hamiltonian is hermitian:", is_hermitian(H))

(13, 13)
Hamiltonian is hermitian: True


In [12]:
Hw, Hv = eigen(H)
print("Spectrum =")
print(Hw)

Spectrum =
[-1.00071764 -0.06689261  0.33995045  0.5         0.5         0.78139303
  1.06367813  1.5         1.5         1.70560167  1.80842232  2.57989791
  2.58866673]


### Bonus

What are the Gauss' law, physical configurations, and the Hamiltonian using open boundary condition?

Construct the Hamiltonian in the full Hilbert space and see if you get the same ground state.